In [1]:
import argparse

class Args(argparse.Namespace):
    config_file='khang_net/configs/yolof/yolof_resnet_50_1x.py'
    eval_only=False
    num_gpus=1
    num_machines=1
    resume=False

args = Args()

In [3]:
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate

"""
cfg = LazyConfig.load("khang_net/configs/yolof/yolof_resnet_50_1x.py")
cfg.train.device = 'mps'
cfg.dataloader.evaluator.dataset_name = 'kitti_val'
cfg.dataloader.train.dataset.names = 'kitti_train'

cfg.dataloader.train.total_batch_size = 2"""

'\ncfg = LazyConfig.load("khang_net/configs/yolof/yolof_resnet_50_1x.py")\ncfg.train.device = \'mps\'\ncfg.dataloader.evaluator.dataset_name = \'kitti_val\'\ncfg.dataloader.train.dataset.names = \'kitti_train\'\n\ncfg.dataloader.train.total_batch_size = 2'

In [3]:
from detectron2.engine import default_setup

In [4]:
default_setup(cfg, args)

[03/11 10:38:17 detectron2]: Rank of current process: 0. World size: 1
[03/11 10:38:18 detectron2]: Environment info:
-------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sys.platform                     darwin
Python                           

In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

dataset = 'kitti'
annot_dir = './kitti/training_coco'
imgs_dir = './kitti/training_voc/VOC2012/JPEGImages'

for split in ['train', 'val']: 
    annot_path = os.path.join(annot_dir, f'{split}.json')
    d_name = dataset + f'_{split}'
    register_coco_instances(d_name, {}, annot_path, imgs_dir)

In [6]:
# Load dataset
dataset_dicts = DatasetCatalog.get('kitti_train')
metadata = MetadataCatalog.get('kitti_train')

[03/11 10:38:18 d2.data.datasets.coco]: Loaded 5984 images in COCO format from ./kitti/training_coco/train.json
[03/11 10:38:18 d2.data.datasets.coco]: Loaded 5984 images in COCO format from ./kitti/training_coco/train.json


In [7]:
cfg.train.device

'mps'

In [9]:
from tools.lazyconfig_train_net import do_train

# Uncomment to train
#do_train(args, cfg)

In [20]:
yolof = instantiate(cfg.model)

In [24]:
for x in yolof.anchor_generator.cell_anchors:
    print(x)


tensor([[ -90.5097,  -45.2548,   90.5097,   45.2548],
        [ -64.0000,  -64.0000,   64.0000,   64.0000],
        [ -45.2548,  -90.5097,   45.2548,   90.5097],
        [-114.0350,  -57.0175,  114.0350,   57.0175],
        [ -80.6349,  -80.6349,   80.6349,   80.6349],
        [ -57.0175, -114.0350,   57.0175,  114.0350],
        [-143.6751,  -71.8376,  143.6751,   71.8376],
        [-101.5937, -101.5937,  101.5937,  101.5937],
        [ -71.8376, -143.6751,   71.8376,  143.6751]])


# Model

In [1]:
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate

cfg = LazyConfig.load("khang_net/configs/yolof/yolof_resnet_50_1x.py")
cfg.train.device = 'mps'


In [2]:
yolof = instantiate(cfg.model)

In [3]:
yolof.device

device(type='cpu')

In [4]:
yolof.anchor_generator.base_anchors.device

device(type='cpu')

In [5]:
yolof.to(cfg.train.device)

YOLOF(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
      )
      (1): BottleneckBlock(
        (

In [6]:
yolof.anchor_generator.base_anchors.device

device(type='mps', index=0)

In [10]:
yolof.anchor_generator

YOLOFAnchorGenerator(
  (list_anchors): BufferList()
)

# Anchor generator

In [11]:
from khang_net.modeling.anchor_generator import YOLOFAnchorGenerator

In [12]:
anchor_generator = YOLOFAnchorGenerator(sizes=(32., 64., 128., 256., 512.),
                                        aspect_ratio=1.,
                                        stride=32,
                                        offset=0.0)

In [13]:
anchor_generator.base_anchors.device

device(type='cpu')

In [14]:
import torch

batch_size = 2
feature_w = feature_h = 1024 // 32
channels = 64

feature_map = torch.randn(batch_size, channels, feature_w, feature_h)

In [16]:
anchors = anchor_generator(feature_map)

In [18]:
anchors.tensor.shape

torch.Size([5120, 4])

In [19]:
from detectron2.structures import Boxes

In [21]:
from detectron2.layers import cat, ciou_loss, diou_loss

In [ ]:
if isinstance(anchors[0], Boxes):
        anchors = type(anchors[0]).cat(anchors).tensor  # (R, 4)

In [24]:
type(anchors)

detectron2.structures.boxes.Boxes

# Train

In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

In [2]:
dataset = 'kitti'
annot_dir = './kitti/training_coco'
imgs_dir = './kitti/training_voc/VOC2012/JPEGImages'

for split in ['train', 'val']: 
    annot_path = os.path.join(annot_dir, f'{split}.json')
    d_name = dataset + f'_{split}'
    register_coco_instances(d_name, {}, annot_path, imgs_dir)

# Load dataset
dataset_dicts = DatasetCatalog.get('kitti_train')
metadata = MetadataCatalog.get('kitti_train')

[03/11 23:25:44 d2.data.datasets.coco]: Loaded 5984 images in COCO format from ./kitti/training_coco/train.json


In [3]:
import argparse

class Args(argparse.Namespace):
    config_file='khang_net/configs/yolof/base_yolof.py'
    eval_only=False
    num_gpus=1
    num_machines=1
    resume=False

args = Args()

In [4]:
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate


cfg = LazyConfig.load("khang_net/configs/yolof/yolof_resnet_50_1x.py")
cfg.train.device = 'mps'
cfg.dataloader.evaluator.dataset_name = 'kitti_val'
cfg.dataloader.train.dataset.names = 'kitti_train'

In [6]:
from detectron2.engine import default_setup

default_setup(cfg, args)

[03/11 23:26:12 detectron2]: Rank of current process: 0. World size: 1
[03/11 23:26:12 detectron2]: Rank of current process: 0. World size: 1
[03/11 23:26:12 detectron2]: Environment info:
-------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sy

In [7]:
from tools.lazyconfig_train_net import do_train

do_train(args, cfg)

[03/11 23:26:28 detectron2]: Model:
YOLOF(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
      )
 

TypeError: conv2d() received an invalid combination of arguments - got (dict, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!dict!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!dict!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
